# 1. Tokenisation

In deze notebook laden we de commentaren van de enquête in en passen we tokenisation toe: het splitsen van de tekst in aparte woorden/entiteiten.

## 1.1 Imports

We laden uitbreidingen in die we nodig hebben om de dataset in te laden.

In [99]:
import pandas as pd
from spacy_download import load_spacy
import os

## 1.2 Dataset inladen

We laden de dataset in en tonen een preview.

In [2]:
df = pd.read_csv("data/data_anon.csv")

In [3]:
df.head()

,Unnamed: 0,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,UserLanguage,Q1,Q2_1,Q2_2,...,Q11num,Q11numopp,Q7binary,Q13dialoog,Q13educatief,Q13elitair,Q13ouderwets,Q13reflectie,Q13fundament,Q13anders
0,1,100,1030,True,2022-05-19 01:49:27,R_2wjiSHblELDyZMY,NL,"Ik ben zestien of ouder, besef dat mijn gegeve...",patricia de martelaere,connie palmen - de vriendschap,...,2.0,0.0,Analoog,een aanzet tot dialoog over literatuur en lite...,0,0,0,een aanzet tot reflectie en een bron voor debat,een fundament van een collectief cultureel geh...,0
1,2,100,252,True,2022-05-19 02:05:02,R_1I6pSyFmgXFtRjH,NL,"Ik ben zestien of ouder, besef dat mijn gegeve...",De Laatste Dichters - Christine Otten,NaN,...,2.0,0.0,NaN,0,0,0,0,0,een fundament van een collectief cultureel geh...,0
2,3,100,651,True,2022-05-19 02:05:56,R_2OVxUcnSQ8BFaqa,NL,"Ik ben zestien of ouder, besef dat mijn gegeve...",Gangreen 1- Jef Geeraerts,De Zondvloed - Jeroen Brouwers,...,2.0,0.0,Analoog,een aanzet tot dialoog over literatuur en lite...,een educatief instrument,0,0,0,een fundament van een collectief cultureel geh...,0
3,4,100,1901,True,2022-05-19 02:31:21,R_2b0CIkQZZJq6ey6,NL,"Ik ben zestien of ouder, besef dat mijn gegeve...",De Kapellekensbaan - Louis Paul Boon,Max Havelaar - Multatuli,...,2.0,0.0,Analoog,0,0,elitair en niet-inclusief,0,0,een fundament van een collectief cultureel geh...,0
4,5,100,440,True,2022-05-19 03:15:49,R_6M3bVJZ8xxgVL5T,NL,"Ik ben zestien of ouder, besef dat mijn gegeve...",Van den vos Reynaerde,Max Havelaar,...,1.0,1.0,Analoog,0,0,0,0,0,een fundament van een collectief cultureel geh...,0


## 1.3 Tokenisation

[spaCy](https://spacy.io/) is een bekende package om tekst te parsen en verwerken. Met spaCy kan je héél veel (POS-taggen, tokenisen, lemmatisen, named entity recognition). Wij gaan gewoon alles opsplitsen in woordjes.

Ik heb [load_spacy](https://github.com/BramVanroy/spacy_download) van collega Bram Vanroy gebruikt, omdat dit automatisch ook het model downloadt als je het nog niet hebt. Verder werkt alles zoals je zou verwachten van spaCy.

In [4]:
nlp = load_spacy("nl_core_news_lg", exclude=["parser", "tagger"])
nlp.add_pipe("sentencizer")
nlp

Ik toon hoe je een zin kunt tokeniseren met spaCy:

In [5]:
doc = nlp("Freek Van de Velde neemt geen deel aan de fantastische taalkundequiz. Ik speel graag Minecraft.")
for sentence_number, sentence in enumerate(doc.sents):
    for token in sentence:
        print(sentence_number, token.text, token.lemma_, token.pos_)

0 Freek Freek PROPN
0 Van Van PROPN
0 de de PROPN
0 Velde Velde PROPN
0 neemt toenemen VERB
0 geen geen DET
0 deel deel NOUN
0 aan aan ADP
0 de de DET
0 fantastische fantastisch ADJ
0 taalkundequiz taalkundequiz NOUN
0 . . PUNCT
1 Ik ik PRON
1 speel speel VERB
1 graag graag ADV
1 Minecraft Minecraft PROPN
1 . . PUNCT


Nu is het de bedoeling dat we hetzelfde doen met de commentaren uit de enquête. Ik geef een voorzet:

In [65]:
rows = []

# We itereren over elke rij van de dataset
for index, row in df.iterrows():
    # pd.isnull wordt gebruikt om lege rijen weg te laten.
    if pd.isnull(row["Q9_ELAB"]):
        continue
    
    #Iedere zin per respondent wordt opgesplitst en benummerd.
    doc = nlp(row["Q9_ELAB"])
    for sentence_number, sentence in enumerate(doc.sents):
        for token in sentence:
    # Je kunt een kolom opvragen door die te indexeren
    # Hier worden de zinnen opgesplitst per respondent.
            new_row = { "response_id": row["ResponseId"], "sentence_no": sentence_number, "token": token.text, "lemma": token.lemma_, "pos": token.pos_ }
            rows.append(new_row)

    # Tokenisation

We willen voor elke token een aparte rij in onze dataset. Zoiets:

|sentence_no|token|lemma|pos|
|---|---|---|---|
|0|Freek|Freek|PROPN|
|0|neemt|nemen|VERB|
|0|deel|deel|NOUN|

Je kunt als volgt een eigen dataset maken:

Je ziet hoe je deze snippet in de loop hierboven kunt zetten om zo een nieuwe dataset te maken.

In [7]:
new_df = pd.DataFrame.from_dict(rows)
new_df.head()

,response_id,sentence_no,token,lemma,pos
0,R_2wjiSHblELDyZMY,0,zonder,zonder,ADP
1,R_2wjiSHblELDyZMY,0,canon,canon,NOUN
2,R_2wjiSHblELDyZMY,0,kan,kunnen,AUX
3,R_2wjiSHblELDyZMY,0,je,je,PRON
4,R_2wjiSHblELDyZMY,0,niks,niks,PRON


De voorgaande code werd aangepast om voor iedere open vraag een nieuwe output-file te maken waarin de open antwoorden gelemmatiseerd wordt.

In [121]:
# Deze lijn sorteert alle toelichtingsvragen uit de dataset en creëert een variabele per open vraag.
ELAB = {question for question in df if "ELAB" in question}
ELAB.add("QPERS_IDEAS")
ELAB.add("QPERS_COMPLAIN")
# De loop wordt gebruikt om iedere vraag apart te analyseren.
for question in ELAB:
    # Door de rijen in de loop te steken, worden de rijen per vraag herschreven.
    rows = []
    # Hier worden steeds per vraag de niet-beantwoorde vragen eruit gefilterd.
    for index, row in df.iterrows():
        if pd.isnull(row[str(question)]):
            continue
        # De beantwoorde vragen worden via deze code gelemmatiseerd waarbij een identifyer, de gebruiker, de zin,
        # het woord, het lemma en de part of speech neergeschreven worden in een dataset.
        doc = nlp(row[str(question)])
        for sentence_number, sentence in enumerate(doc.sents):
            for token in sentence:
                if "ELAB" in question:
                    new_row = { "identifyer": question.replace('ELAB', '') + row["ResponseId"] + '_' + str(sentence_number), "response_id": row["ResponseId"], "sentence_number": sentence_number, "token": token.text, "lemma": token.lemma_, "pos": token.pos_ }
                    rows.append(new_row)
                elif "IDEAS" in question:
                    new_row = { "identifyer": question.replace('QPERS_IDEAS', 'QI_') + row["ResponseId"] + '_' +str(sentence_number), "response_id": row["ResponseId"], "sentence_number": sentence_number, "token": token.text, "lemma": token.lemma_, "pos": token.pos_ }
                    rows.append(new_row)
                else:
                    new_row = { "identifyer": question.replace('QPERS_COMPLAIN', 'QC_') + row["ResponseId"] + '_' +str(sentence_number), "response_id": row["ResponseId"], "sentence_number": sentence_number, "token": token.text, "lemma": token.lemma_, "pos": token.pos_ }
                    rows.append(new_row)
    # De loop maakt verschillende .csv-bestanden aan waarbij de lemmatisering van de vragen respectievelijk opgeslaan worden.
    new_df = pd.DataFrame.from_dict(rows) 
    new_df.to_csv("sorted_data/" + question + ".csv")

De afzonderlijke documenten worden via deze code gecombineerd, aan de hand van de identifyer (bestaande uit question-number, ResponseId en sentence number) om de vragen uit elkaar te houden.

In [123]:
c_path = 'sorted_data/'
csv_all = os.listdir(c_path)
df_list = []

for csv in csv_all:
    csv_path = os.path.join(c_path, csv)
    try:
        df = pd.read_csv(c_path)
        df_list.append(df)
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(c_path, sep='/t', encoding='utf-16')
            df_list.append(df)
        except Exception as e:
            print(f"Could not read file {csv} because of error: {e}")
    except Exception as e:
        print(f"Could not read file {csv} because of error: {e}")
                            
big_df = pd.concat(df_list)

big_df.to_csv(os.path.join(c_path, 'Combined_ELAB.csv'))

Could not read file Q10_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q11_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q14_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q15_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q16_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q17_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file Q9_ELAB.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file QPERS_COMPLAIN.csv because of error: [Errno 13] Permission denied: 'sorted_data/'
Could not read file QPERS_IDEAS.csv because of error: [Errno 13] Permission denied: 'sorted_data/'


ValueError: No objects to concatenate

De canonenquête bevatte 7 toelichtingsvragen:
- Q9: De aandacht voor de canon in voortgezet/secundair onderwijs.
- Q10: Leerlingen in het voortgezet/secundair onderwijs moeten enkele klassiekers lezen.
- Q11: Leerlingen in het voortgezet/secundair onderwijs moeten een elementaire kennis aangeboden krijgen van Nederlandstalige literatuurgeschiedenis
- Q14: Een Nederlandstalige literaire canon moet streven naar genderdiversiteit.
- Q15: Er kunnen meerdere Nederlandstalige literaire canons naast elkaar bestaan.
- Q16: Welke teksten uit de voormalige koloniale gebieden zouden in de canon moeten?
- Q17: Welke teksten uit de kinder- en jeugdliteratuur zouden in de canon moeten?